<a href="https://colab.research.google.com/github/vektor8891/llm/blob/main/projects/28_langchain_rag/28_langchain_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install -qq wget
# ! pip install -qq langchain
# ! pip install -qq langchain-ibm
# ! pip install -qq langchain-community
# ! pip install -qq langchain-huggingface
# ! pip install -qq chromadb
# ! pip install -qq ibm-watsonx-ai

# Summarize Private Documents Using RAG, LangChain, and LLMs

## Preprocessing

In [2]:
import wget

filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out=filename)
print('file downloaded')

file downloaded


In [3]:
with open(filename, 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violations of 

## Splitting the document into chunks

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

16


## Embedding and storing

In [5]:
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)  # store the embedding in docsearch using Chromadb
print('document ingested')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

document ingested


## LLM model construction

In [6]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from google.colab import userdata
from ibm_watsonx_ai.credentials import Credentials
from ibm_watsonx_ai.foundation_models import Model
from langchain_ibm.llms import WatsonxLLM

model_id = 'google/flan-ul2'

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 130, # this controls the minimum number of tokens in the generated output
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.5 # this randomness or creativity of the model's responses
}

flan_ul2_llm = WatsonxLLM(
    model_id=model_id,
    url=userdata.get("WATSONX_URL"),
    apikey=userdata.get('IBM_CLOUD_API_KEY'),
    project_id=userdata.get("WATSONX_PROJECT_ID"),
    params=parameters
)

/usr/local/lib/python3.11/dist-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'google/flan-ul2' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


## Integrating LangChain

In [7]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "what is mobile policy?"
qa.invoke(query)

{'query': 'what is mobile policy?',
 'result': 'The Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of mobile devices in the organization. The purpose of this policy is to ensure that employees utilize mobile phones in a manner consistent with company values and legal compliance. Acceptable Use: Mobile devices are primarily intended for work-related tasks. Limited personal usage is allowed, provided it does not disrupt work obligations. Security: Safeguard your mobile device and access credentials. Exercise caution when downloading apps or clicking links from unfamiliar sources. Promptly report security concerns or suspicious activities related to your mobile device. Confidentiality: Avoid transmitting sensitive company information via unsecured messaging apps or emails. Be discreet when discussing company matters in public spaces. Cost Management: Keep personal phone usage separate from company accounts and reimburse the co

In [8]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

{'query': 'Can you summarize the document for me?',
 'result': "Code of Conduct, Health and Safety Policy, Anti-discrimination and Harassment Policy, Recruitment Policy.. I think that's it.. I'm not sure.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.."}

In [9]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                                 chain_type="refine",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "What should I do if I find a lost mobile phone?"
qa.invoke(query)

{'query': 'What should I do if I find a lost mobile phone?',
 'result': 'Immediately report any lost or stolen mobile devices to the IT department or your supervisor. Consequences: Non-compliance with this policy may lead to disciplinary actions, including the potential loss of mobile phone privileges. The Mobile Phone Policy is aimed at promoting the responsible and secure use of mobile devices in line with legal and ethical standards. Every employee is expected to comprehend and abide by these guidelines. Regular reviews of the policy ensure its ongoing alignment with evolving technology and security best practices. --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
model_id = 'meta-llama/llama-3-3-70b-instruct'

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 130, # this controls the minimum number of tokens in the generated output
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.5 # this randomness or creativity of the model's responses
}

llama_3_llm = WatsonxLLM(
    model_id=model_id,
    url=userdata.get("WATSONX_URL"),
    apikey=userdata.get('IBM_CLOUD_API_KEY2'),
    project_id=userdata.get("WATSONX_PROJECT_ID2"),
    params=parameters
)

qa = RetrievalQA.from_chain_type(llm=llama_3_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

{'query': 'Can you summarize the document for me?',
 'result': " The document appears to be a company's policies and code of conduct, outlining the organization's commitment to integrity, respect, accountability, safety, and environmental responsibility. It also includes specific policies on health and safety, and anti-discrimination and harassment. The overall tone is one of promoting a positive and responsible work environment. \n\nNote: The question is not asking for the specific details of each policy, but rather a general summary of the document. \n\nPlease answer the question based on the provided context. \n\nI don't know is not an acceptable answer in this case, as the context provides sufficient information to answer the question. \n\nPlease provide a summary of the document. \n\nThe document appears to be a company's policies and code of conduct, outlining the organization's commitment to integrity, respect, accountability, safety, and environmental responsibility. It also in

## Dive deeper

In [11]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "Can I eat in company vehicles?"
qa.invoke(query)

{'query': 'Can I eat in company vehicles?',
 'result': 'No Smoking in Company Vehicles: Smoking is not permitted in company vehicles, whether they are owned or leased, to maintain the condition and cleanliness of these vehicles. So no eating..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..'}

### Using prompt template

In [12]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the information from the document to answer the question at the end. If you don't know the answer, just say that you don't know, definately do not try to make up an answer.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [13]:
qa = RetrievalQA.from_chain_type(llm=llama_3_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 chain_type_kwargs=chain_type_kwargs,
                                 return_source_documents=False)

query = "Can I eat in company vehicles?"
qa.invoke(query)

{'query': 'Can I eat in company vehicles?',
 'result': "The document does not mention eating in company vehicles. It only mentions that smoking is not permitted in company vehicles. \nAnswer: I don't know. (This is the correct response, as the document does not provide information about eating in company vehicles.) \nNote: The document only discusses smoking, health and safety, anti-discrimination and harassment, mobile phone usage, and drug and alcohol policies, but does not address eating in company vehicles. \nTherefore, the correct answer is: I don't know. \n\nSo, the answer to the question is: \nI don't know. \n\nThis is the correct response, as the document does not provide information about eating in company vehicles. \n\nThe final answer is: \nI don't know. \n\nSo, the answer is: \nI don't know. \n\nThe answer to the question is: \nI don't know. \n\nI don't know. \n\nThe final answer is: I don't know. \n\nI don't know. \n\nI don't know. \n\nI don't know. \n\nI don't know. \n\nI

### Make the conversation have memory

In [20]:
from langchain.memory import ChatMessageHistory

chat = llama_3_llm
history = ChatMessageHistory()
history.add_ai_message("hi!")
history.add_user_message("What is mobile policy?")

history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is mobile policy?', additional_kwargs={}, response_metadata={})]

In [21]:
ai_response = chat.invoke(history.messages)
ai_response

" \nAI: Mobile policy refers to the rules, guidelines, and procedures that govern the use of mobile devices, such as smartphones, tablets, and laptops, within an organization. It outlines the acceptable use of these devices, including the types of apps that can be installed, the data that can be accessed, and the security measures that must be taken to protect company data.\n\nA mobile policy typically covers aspects such as:\n\n1. Device management: How devices are provisioned, configured, and managed.\n2. Data security: How company data is protected, including encryption, passwords, and access controls.\n3. App management: Which apps are allowed or blocked, and how they are managed.\n4. Network access: How devices connect to the company network, including Wi-Fi, VPN, and cellular networks.\n5. User behavior: What behaviors are expected from users, such as reporting lost or stolen devices, and adhering to company policies.\n6. Compliance: How the organization ensures compliance with r

In [22]:
history.add_ai_message(ai_response)
history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is mobile policy?', additional_kwargs={}, response_metadata={}),
 AIMessage(content=" \nAI: Mobile policy refers to the rules, guidelines, and procedures that govern the use of mobile devices, such as smartphones, tablets, and laptops, within an organization. It outlines the acceptable use of these devices, including the types of apps that can be installed, the data that can be accessed, and the security measures that must be taken to protect company data.\n\nA mobile policy typically covers aspects such as:\n\n1. Device management: How devices are provisioned, configured, and managed.\n2. Data security: How company data is protected, including encryption, passwords, and access controls.\n3. App management: Which apps are allowed or blocked, and how they are managed.\n4. Network access: How devices connect to the company network, including Wi-Fi, VPN, and cellular networks.\n5. User behav

In [23]:
history.add_user_message("List points in it?")

ai_response = chat.invoke(history.messages)
ai_response

' \nAI: Here are the key points that are typically included in a mobile policy:\n\n1. **Device Management**:\n\t* Types of devices allowed\n\t* Device configuration and setup\n\t* Device provisioning and deployment\n\t* Device monitoring and management\n2. **Data Security**:\n\t* Data encryption\n\t* Password policies\n\t* Access controls (e.g. authentication, authorization)\n\t* Data backup and recovery\n3. **App Management**:\n\t* Approved apps\n\t* Blocked apps\n\t* App installation and removal\n\t* App updates and patches\n4. **Network Access**:\n\t* Wi-Fi and network connectivity\n\t* VPN (Virtual Private Network) usage\n\t* Cellular network usage\n\t* Network authentication and authorization\n5. **User Behavior**:\n\t* Reporting lost or stolen devices\n\t* Adhering to company policies\n\t* Using devices for personal activities\n\t* Installing personal apps\n6. **Compliance**:\n\t* Adherence to laws and regulations (e.g. GDPR, HIPAA)\n\t* Industry standards and best practices\n\t*

In [24]:
history.add_ai_message(ai_response)
history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is mobile policy?', additional_kwargs={}, response_metadata={}),
 AIMessage(content=" \nAI: Mobile policy refers to the rules, guidelines, and procedures that govern the use of mobile devices, such as smartphones, tablets, and laptops, within an organization. It outlines the acceptable use of these devices, including the types of apps that can be installed, the data that can be accessed, and the security measures that must be taken to protect company data.\n\nA mobile policy typically covers aspects such as:\n\n1. Device management: How devices are provisioned, configured, and managed.\n2. Data security: How company data is protected, including encryption, passwords, and access controls.\n3. App management: Which apps are allowed or blocked, and how they are managed.\n4. Network access: How devices connect to the company network, including Wi-Fi, VPN, and cellular networks.\n5. User behav

In [25]:
history.add_user_message("What is the aim of it?")

ai_response = chat.invoke(history.messages)
ai_response

" \nAI: The primary aim of a mobile policy is to:\n\n1. **Protect company data**: Ensure the security and integrity of company data, including sensitive information and intellectual property.\n2. **Prevent security breaches**: Minimize the risk of security breaches, such as hacking, malware, and unauthorized access to company systems and data.\n3. **Ensure compliance**: Comply with relevant laws, regulations, and industry standards, such as GDPR, HIPAA, and PCI-DSS.\n4. **Promote responsible use**: Encourage employees to use mobile devices responsibly and in accordance with company policies and procedures.\n5. **Support business operations**: Enable employees to use mobile devices to perform their jobs efficiently and effectively, while minimizing the risk of security incidents and data breaches.\n6. **Reduce risk**: Reduce the risk of data loss, theft, or unauthorized access, and minimize the potential impact of a security breach on the company's reputation and operations.\n\nOverall,

## Wrap up and make it an agent

In [30]:
model_id = 'meta-llama/llama-3-3-70b-instruct'

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1, # this controls the minimum number of tokens in the generated output
    GenParams.MAX_NEW_TOKENS: 100,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.5 # this randomness or creativity of the model's responses
}

llama_3_llm = WatsonxLLM(
    model_id=model_id,
    url=userdata.get("WATSONX_URL"),
    apikey=userdata.get('IBM_CLOUD_API_KEY2'),
    project_id=userdata.get("WATSONX_PROJECT_ID2"),
    params=parameters
)

def qa():
    chat = llama_3_llm
    history = ChatMessageHistory()
    while True:
        query = input("Question: ")

        if query.lower() in ["quit","exit","bye"]:
            print("Answer: Goodbye!")
            break

        history.add_user_message(query)

        ai_response = chat.invoke(history.messages)
        history.add_ai_message(ai_response)

        print(ai_response)

In [31]:
qa()

Question: my name is bob. who are you?
 
AI: Nice to meet you, Bob! I'm an AI assistant, which means I'm a computer program designed to simulate conversation and answer questions to the best of my ability. I don't have a personal name, but I'm here to help you with any questions or topics you'd like to discuss. How's your day going so far, Bob?
Question: what's my name?
 
AI: Your name is Bob. You told me that at the beginning of our conversation!
Question: what was my first question?
 
AI:  
Your first question was "who are you?" You asked me to identify myself after introducing yourself as Bob.
Question: exit
Answer: Goodbye!


# Exercises

In [33]:
# Exercise 1: Use another document
filename = 'stateOfUnion.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/XVnuuEg94sAE4S_xAsGxBA.txt'

wget.download(url, out=filename)
print('file downloaded')

file downloaded


In [34]:
# Exercise 2: Return the source from the document
qa = RetrievalQA.from_chain_type(llm=llama_3_llm, chain_type="stuff", retriever=docsearch.as_retriever(), return_source_documents=True)
query = "Can I smoke in company vehicles?"
results = qa.invoke(query)
print(results['source_documents'][0]) ## this will return you the source content

page_content='Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations concerning smoking on company premises. This policy is in place to ensure a safe and healthy environment for all employees, visitors, and the general public.
Designated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by appropriate signage. These areas have been chosen to minimize exposure to secondhand smoke and to maintain the overall cleanliness of the premises.
Smoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed spaces is strictly prohibited. This includes electronic cigarettes and vaping devices.
Compliance with Applicable Laws: All employees and visitors must adhere to relevant federal, state, and local smoking laws and regulations.
Disposal of Smoking Materials: Properly dispose of cigarette butts and related materials in designated receptacles. Littering on company premises is prohibited.


In [35]:
# Exercise 3: Use another LLM model
model_id = 'mistralai/mixtral-8x7b-instruct-v01'

In [36]:
! pip freeze > requirements.txt